In [14]:
import networkx as nx
import numpy as np
from random import random
from random import choice
from pickle import load
import os

path = os.getcwd()


In [18]:
def get_graph(s,K,a,pi,sigma):

    dist = {k:a[k,s] for k in K if pi[k]>0}
    Ks = [k for k in sorted(dist, key=dist.get, reverse=True)]
    #print(Ks)

    V = ["e"] + Ks + ["s"]
    A = [(i,j) for i in V for j in V if  i!=j and i!="e" and j!="s" and a[i,s] <= a[j,s] and (i,j)!=("s","e")]

    rc = {arc:-pi[arc[1]]-sigma if arc[0]=="s" else (0 if arc[1]=="e" else -pi[arc[1]]) for arc in A}
    
    return V,A,rc

def get_vertices_successors(V,A):
    
    G = nx.DiGraph()
    G.add_nodes_from(V); G.add_edges_from(A)
    successors = {}
    for v in V:
        successors[v] = list(G.successors(v))

    return successors

def sort_successors(v,tP,successors,s,a,t):

    if v == "s": nodes = successors
    else: nodes = successors[1:]
    fin_times = np.array([np.max((tP,a[i,s]))+t[i,s] for i in nodes])
    sorted_indices = np.argsort(fin_times)
    sort_array = np.array(nodes)[sorted_indices].tolist()
    if v not in ["s","e"]: sort_array.append("e")

    return sort_array

def disjunctive_routes(K,s,a,t,pi,sigma,y):
    routes = list()
    unassigned_K = [k for k in K]
    while len(unassigned_K) > 0 and len(routes) < y:
        V,A,rc = get_graph(s,unassigned_K,a,pi,sigma)
        successors = get_vertices_successors(V,A)
        opt_route = labeling(V,rc,successors,s,a,t)
        if opt_route["r"] < -0.001: print(f"New route found: {len(opt_route['P'][1:]),opt_route['P'][1:]}"); routes.append(opt_route["P"][1:])
        else: break
        for k in opt_route["P"][1:]:
            unassigned_K.remove(k)
    return routes

def labeling(V,rc,successors,s,a,t):
    r_bound = {v:0 for v in V}; P_bound = {v:list() for v in V}
    for v in V[1:]:
        P = list(); opt_route = {"r":1e9, "P":list()}
        rP, tP, qP = 0, a[v,s]+t[v,s], a[v,s]
        pulse(v,P,rP,tP,qP,opt_route,r_bound,P_bound,rc,successors,s,a,t)
        r_bound[v] = opt_route["r"]; P_bound[v] = opt_route["P"]

    return opt_route

def pulse(v,P,rP,tP,qP,opt_route,r_bound,P_bound,rc,successors,s,a,t):

    ''' Check label feasibility '''
    if v not in P and a[v,s] >= qP:
        #print(f"Extension {P+[v]} if feasible")
        ''' Check bounds '''
        if rP + r_bound[v] < opt_route["r"]:
            #print(f"Extension {P+[v]} is not dominated")
            
            ''' Update optimal route if pulse reached sink vertex'''
            if v == "e" and rP < opt_route["r"]:
                opt_route["r"] = rP; opt_route["P"] = P

            ''' Update resources consumption '''
            if a[v,s] == tP - t[v,s] and len(P) > 0:

                new_P = P + P_bound[v]
                new_qP = qP
                new_rP = rP + r_bound[v]
                #print(f"Vehicle {v} started on time in route {P}",new_P)
                pulse("e",new_P,new_rP,tP,qP,opt_route,r_bound,P_bound,rc,successors,s,a,t)
            
            else:
                new_P = P + [v]
                new_qP = tP - t[v,s]

                ''' Extend pulse '''
                sorted_succesors = sort_successors(v,tP,successors[v],s,a,t)
                for v1 in sorted_succesors:
                    new_rP = rP + rc[v,v1]
                    new_tP = np.max((tP,a[v1,s])) + t[v1,s]
                    pulse(v1,new_P,new_rP,new_tP,new_qP,opt_route,r_bound,P_bound,rc,successors,s,a,t)


In [19]:
sc = 1; s = 379

file = open(path+f"/Data/K_s10/Ks_sc{sc}","rb")
K_s = load(file); file.close()

file = open(path+f"/Data/a t/at_{sc}","rb")
a, t = load(file); file.close()

a.update({("s",s):0,("e",s):40})
t.update({("s",s):0,("e",s):0})

pi = {k:10 for k in K_s[s]}; sigma = 0

In [20]:
routes = disjunctive_routes(K_s[s],s,a,t,pi,sigma,8)
routes

New route found: (19, [9120, 6996, 2858, 6939, 10392, 8028, 6962, 8907, 3549, 2629, 10373, 7174, 4725, 8891, 7155, 10411, 9220, 8234, 2555])
New route found: (17, [5026, 7892, 3618, 1601, 4920, 9236, 7896, 1354, 9350, 4997, 7958, 2827, 2478, 1600, 397, 5776, 6949])
New route found: (16, [2607, 7038, 10054, 3947, 2665, 1338, 1639, 9992, 9360, 8975, 266, 288, 5654, 3686, 9198, 8253])
New route found: (14, [1554, 9317, 10213, 2646, 590, 7841, 6839, 6883, 10055, 259, 9328, 6794, 8913, 6755])
New route found: (13, [9977, 2680, 7092, 10076, 6749, 2439, 3827, 6804, 564, 6105, 10299, 8997, 6817])
New route found: (13, [4636, 365, 8177, 2679, 522, 4591, 4679, 8060, 1703, 9313, 6733, 5672, 8113])
New route found: (12, [471, 2782, 1345, 1764, 10186, 381, 7830, 1683, 7834, 2839, 3496, 3518])
New route found: (12, [330, 3642, 1442, 1544, 5955, 8944, 7828, 6095, 6871, 1488, 521, 3933])


[[9120,
  6996,
  2858,
  6939,
  10392,
  8028,
  6962,
  8907,
  3549,
  2629,
  10373,
  7174,
  4725,
  8891,
  7155,
  10411,
  9220,
  8234,
  2555],
 [5026,
  7892,
  3618,
  1601,
  4920,
  9236,
  7896,
  1354,
  9350,
  4997,
  7958,
  2827,
  2478,
  1600,
  397,
  5776,
  6949],
 [2607,
  7038,
  10054,
  3947,
  2665,
  1338,
  1639,
  9992,
  9360,
  8975,
  266,
  288,
  5654,
  3686,
  9198,
  8253],
 [1554,
  9317,
  10213,
  2646,
  590,
  7841,
  6839,
  6883,
  10055,
  259,
  9328,
  6794,
  8913,
  6755],
 [9977,
  2680,
  7092,
  10076,
  6749,
  2439,
  3827,
  6804,
  564,
  6105,
  10299,
  8997,
  6817],
 [4636, 365, 8177, 2679, 522, 4591, 4679, 8060, 1703, 9313, 6733, 5672, 8113],
 [471, 2782, 1345, 1764, 10186, 381, 7830, 1683, 7834, 2839, 3496, 3518],
 [330, 3642, 1442, 1544, 5955, 8944, 7828, 6095, 6871, 1488, 521, 3933]]

In [22]:
sc = 0; s = 379

file = open(path+f"/Data/K_s10/Ks_sc{sc}","rb")
K_s = load(file); file.close()

file = open(path+f"/Data/a t/at_{sc}","rb")
a, t = load(file); file.close()

a.update({("s",s):0,("e",s):40})
t.update({("s",s):0,("e",s):0})

pi = {k:10 for k in K_s[s]}; sigma = 0

In [23]:
routes_other = disjunctive_routes(K_s[s],s,a,t,pi,sigma,8)

New route found: (19, [8898, 1779, 10277, 5707, 6762, 10288, 6740, 2607, 4725, 8075, 6749, 2854, 1528, 465, 8117, 7934, 1764, 2868, 8281])


KeyboardInterrupt: 